In [ ]:
from sliderule import sliderule, icesat2, earthdata
from shapely.geometry import Polygon, MultiPolygon, LineString, mapping
import geopandas as gpd
import geojson

### Read pond time,lat,lon into GeoDataFrame

In [ ]:
df = gpd.pd.read_csv("MysteryLake_radius100m.csv")

In [ ]:
geometry = gpd.points_from_xy(df.Longitude, df.Latitude)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

In [ ]:
gdf

#### Create buffered polygon of linestring of lat,lon list

In [ ]:
# display the original path of the pond
linestring = LineString(list(gdf.geometry))
linestring

In [ ]:
# project to "sea ice polar stereographic north" (units in meters)
proj_gdf = gdf.to_crs("EPSG:3413")
proj_linestring = LineString(list(proj_gdf.geometry))
proj_linestring

In [ ]:
# buffer out 1km to polygon 
proj_polygon = proj_linestring.buffer(1000)
proj_polygon

In [ ]:
# reproject polygon back to WGS84 so that we can query CMR
reproj_gdf = gpd.GeoDataFrame(geometry=[proj_polygon], crs="EPSG:3413")
reproj_gdf = reproj_gdf.to_crs("EPSG:4326")
reproj_gdf

### Use sliderule to get ATL03 granules that intersect path of pond

In [ ]:
# initialize sliderule client (enable verbose)
sliderule.init(verbose=True)

In [ ]:
# create a region of interest out of the path of the pond
region = sliderule.toregion(reproj_gdf)

In [ ]:
# query CMR for ATL03 granules that intersect the path of the pond
earthdata.set_max_resources(500)
cmr_parms = {
    "asset": "icesat2",
    "poly": region["poly"],
    "t0": "2020-05-01",
    "t1": "2020-07-31"
}
atl03_granules = earthdata.search(cmr_parms)
atl03_granules

### Make request to sliderule to provide ATL03 segment information for every segment that intersects with the path of the pond

In [ ]:
# build the processing parameters for a sliderule query to the atl03v endpoint
parms = {
    "poly": region["poly"]
}

In [ ]:
# make sliderule request to atl03v
atl03v = icesat2.atl03vp(parms, resources=atl03_granules)

In [ ]:
atl03v

In [ ]:
# project the results to "sea ice polar stereographic north" (units in meters)
proj_atl03v = atl03v.to_crs("EPSG:3413")

In [ ]:
proj_atl03v.plot(markersize=1)

### Plot the path of the pond overtop the ATL03 segments

In [ ]:
# get the buffered path of the pond in the same projection
proj_buffered_gdf = gpd.GeoDataFrame(geometry=[proj_polygon], crs="EPSG:3413")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f, ax = plt.subplots()
proj_atl03v.plot(ax=ax, color='blue', markersize=1.0)
proj_buffered_gdf.plot(ax=ax, color='red', markersize=1.0)
plt.show()

### Trim the ATL03 segments to exactly the path of the pond

In [ ]:
trimmed_proj_atl03v = proj_atl03v.sjoin(proj_buffered_gdf, how='inner', predicate='within')

In [ ]:
trimmed_proj_atl03v.plot(markersize=1)

In [ ]:
trimmed_proj_atl03v

### Determine every segment in ATL03 that is within 10 minutes and 2 Km of a point in the pond path

In [ ]:
# Create projected pond geodataframe with "Date" as its index
gdf['Date'] = gpd.pd.to_datetime(gdf["Time"])
pond_gdf = gdf.set_index('Date')
proj_pond_gdf = pond_gdf.to_crs("EPSG:3413")

In [ ]:
# calculate which ATL03 segments are close in time and distance to the pond path
import warnings
warnings.filterwarnings('ignore')
trimmed_proj_atl03v["mask"] = 0
hits = 0
hit_dictionary = {}
atl03v = trimmed_proj_atl03v
time_threshold = 10
for i in range(len(proj_pond_gdf.index)):
    
    t_pond = proj_pond_gdf.index[i]
    atl03v["time_delta"] = abs(atl03v.index - t_pond)
    atl03v_close_in_time = atl03v[atl03v["time_delta"] < gpd.pd.Timedelta(f'{time_threshold} minutes')]

    if(len(atl03v_close_in_time) > 0):
        atl03v_close_in_time["distance_delta"] = atl03v_close_in_time.geometry.distance(proj_pond_gdf.geometry.iloc[i])

        for index, row in atl03v_close_in_time.iterrows():
            distance = row["distance_delta"]
            if type(distance) == float:
                distance_km = distance / 1000
            else:
                distance_km = distance.min() / 1000
    
            if distance_km < 2:
                trimmed_proj_atl03v.loc[index,"mask"] = 1
                hits += 1
                hit_dictionary[index] = True
                print(f'Time match found within {row["time_delta"].total_seconds() / 60:.3} minutes: index={index}, distance={distance_km}, cycle={row["cycle"]}, rgt={row["rgt"]}')
print("Total hits: ", hits)
print("Total number of unique indices: ", len(hit_dictionary))

In [ ]:
#
# This is another way to calculate the segments that are close in time and distance,
# but since it loops over all of the segments, it is slower
#
# calculate which ATL03 segments are close in time and distance to the pond path
# trimmed_proj_atl03v["mask"] = 0
# time_threshold = 10
# for i in range(len(trimmed_proj_atl03v.index)):
#     t = trimmed_proj_atl03v.index[i]
    
#     closest_time = abs(proj_pond_gdf.index - t)
#     time_delta = abs(closest_time.min().total_seconds() / 60)
#     if time_delta <= time_threshold:

#         pond_index = closest_time.argmin()
#         pond_geo = proj_pond_gdf.geometry.iloc[pond_index]
#         icesat2_geo = trimmed_proj_atl03v.geometry.iloc[i]
#         distance = pond_geo.distance(icesat2_geo)
        
#         row = trimmed_proj_atl03v.iloc[i]

#         if type(distance) == float:
#             distance_km = distance / 1000
#         else:
#             distance_km = distance.min() / 1000

#         if distance_km < 2:
#             trimmed_proj_atl03v.loc[t,"mask"] = 1
#             print(f'Time match found within {time_delta:.3} minutes: distance={distance_km}, cycle={row["cycle"]}, rgt={row["rgt"]}')

In [ ]:
# pull out only the ATL03 segments that matched the criteria
masked_atl03v = trimmed_proj_atl03v[trimmed_proj_atl03v["mask"] == 1]

In [ ]:
masked_atl03v

In [ ]:
%matplotlib widget
f, ax = plt.subplots()
proj_buffered_gdf.plot(ax=ax, color='red', markersize=1.0)
masked_atl03v.plot(ax=ax, color='blue', markersize=1.0)
plt.show()

### Plot intersections with time as the z-dimension to look for exact intersections

In [ ]:
pond_resampled = proj_pond_gdf[['Latitude','Longitude']].resample('30s').asfreq()
pond_resampled['x'] = proj_pond_gdf.geometry.x
pond_resampled['y'] = proj_pond_gdf.geometry.y
pond_resampled

In [ ]:
pond_interpolated = pond_resampled.interpolate(method='time')

In [ ]:
pond_interpolated.index

In [ ]:
list_of_time_references = [
    '2020-05-09 10:00:36.200328192',
    '2020-06-14 22:57:39.297117696',
    '2020-06-30 08:11:44.577678080',
    '2020-07-11 08:12:17.927471616',
    '2020-07-12 21:59:38.291162880'
]
list_of_dimensions = [
    ((577000, 582000), (-350985, -340985)),
    ((651000, 654000), (-492000, -488000)),
    ((710000, 730000), (-526000, -520000)),
    ((672000, 678000), (-639000, -633000)),
    ((659500, 662000), (-656000, -651000))
]

In [ ]:
import pandas as pd
from matplotlib.patches import Circle
import mpl_toolkits.mplot3d.art3d as art3d

In [ ]:
fig = plt.figure(figsize=(8,24))

for x in range(len(list_of_time_references)):
    ax = fig.add_subplot(5,1,x+1, projection='3d') # sharex=True, sharey=True

    reference_datetime = pd.Timestamp(list_of_time_references[x])
    pond_interpolated['seconds_since_reference'] = (pond_interpolated.index - reference_datetime).total_seconds()
    pond_interpolated_segment = pond_interpolated[abs(pond_interpolated.seconds_since_reference) < 3600]
    masked_atl03v['seconds_since_reference'] = (masked_atl03v.index - reference_datetime).total_seconds()

    
    for i in range(len(pond_interpolated_segment)):
        p = Circle((pond_interpolated_segment.x[i], pond_interpolated_segment.y[i]), 100)
        ax.add_patch(p)
        art3d.pathpatch_2d_to_3d(p, z=pond_interpolated_segment.seconds_since_reference[i], zdir="z")
    
    ax.scatter(masked_atl03v.geometry.x, masked_atl03v.geometry.y, masked_atl03v.seconds_since_reference, color='green') 
    ax.set_xlim(list_of_dimensions[x][0][0], list_of_dimensions[x][0][1])
    ax.set_ylim(list_of_dimensions[x][1][0], list_of_dimensions[x][1][1])
    ax.set_zlim(-4000, 4000)
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_zlabel('relative time [s]')